In [11]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import csv

# Load the training data
df = pd.read_csv('train.csv')

# Encode categorical columns
object_columns = df.select_dtypes(include=['object']).columns
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in object_columns:
    df[col] = le.fit_transform(df[col])

# Separate features (X) and target variable (y)
X = df.drop(['price_doc'], axis=1)
y = df['price_doc']

# Load the test data
df_test = pd.read_csv('test.csv')

# Encode categorical columns in the test set
for col in object_columns:
    df_test[col] = le.fit_transform(df_test[col])

# Drop unnecessary column from the test set
df_test = df_test.drop(['row ID'], axis=1)

# Use statsmodels to perform OLS regression and get p-values
X_ols = sm.add_constant(X)  # Add a constant term for the intercept
ols_model = sm.OLS(y, X_ols).fit()

# Get p-values for each feature
p_values = ols_model.pvalues[1:]  # Exclude the constant term

# Select the top 100 features based on p-values
selected_features = list(p_values.sort_values().head(100).index)


In [12]:

# Load the test data
df_test = pd.read_csv('test.csv')

# Encode categorical columns in the test set
for col in object_columns:
    df_test[col] = le.fit_transform(df_test[col])

# Drop unnecessary column from the test set
df_test = df_test.drop(['row ID'], axis=1)

# Use statsmodels to perform OLS regression and get p-values
X_ols = sm.add_constant(X)  # Add a constant term for the intercept
ols_model = sm.OLS(y, X_ols).fit()

# Get p-values for each feature
p_values = ols_model.pvalues[1:]  # Exclude the constant term

# Select the top 100 features based on p-values
selected_features = list(p_values.sort_values().head(30).index)


In [13]:

# Extract the selected features
X_selected = X[selected_features]
df_test_selected = df_test[selected_features]

# Use PolynomialFeatures to create polynomial features
degree = 2
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X_selected)
df_test_poly = poly.transform(df_test_selected)


In [14]:
# Fit a linear regression model on the polynomial features
reg = LinearRegression().fit(X_poly, y)

# Predict on the test set using the polynomial features
y_pred_test = reg.predict(df_test_poly)

In [15]:
# Save the predictions to a CSV file
filepath = 'prediction_pvalue_filter.csv'
with open(filepath, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['row ID', 'price_doc'])  # Add column headers
    for c, i in enumerate(y_pred_test, start=1):
        writer.writerow([c, i])